Not an actual submission, just supporting calculations for: https://www.kaggle.com/wonkydiamond/redepa-quantifying-good-coverage-but-no-throw

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from joblib import load
from tqdm import tqdm
import time 

In [ ]:
from nflutil import NFLData, PlayData

In [ ]:
path_to_data = '../input/nfl-big-data-bowl-2021/'
data = NFLData(data_path = path_to_data, wk_numb = 1)
df_plays= data.plays_df()
df_games = data.games_df()
df_track = data.track_df(join_plays = True)

In [ ]:
df1 = df_plays[df_plays.week.isin([1,])] ###add more week numbers here, beware of kaggles time limits
print(df1.shape)

In [ ]:
yards_regModel = load('../input/nflmodels/yards_RegModel.joblib')
epa_regModel = load('../input/nflmodels/ep_Model.joblib')
comp_clfModel = load('../input/nflmodels/comp_ProbaModel.joblib')

In [ ]:
t_s = time.time()
current_week = data.wk_numb
collected_stats = []

for ind, row_play in tqdm(df1.iterrows(), total = df1.shape[0],\
                           position=0, leave=True):
    
    if row_play.week != current_week:
        current_week = row_play.week
        print("\n Changing to Week {}".format(current_week))
        print("--- {minutes:.0f}m {seconds:.0f}s ---".format(minutes = (time.time() - t_s)/60,
                                                             seconds = (time.time() - t_s) % 60))
        data = NFLData(data_path = path_to_data, wk_numb = current_week)
        df_track = data.track_df(join_plays = True)

    try: 
        plaV = PlayData(df_plays, df_track, pIndex = ind, 
                        epModel = epa_regModel, compProbaModel = comp_clfModel, estYardsModel = yards_regModel)
        plaVstat = plaV.coverage_agg(add_pIndex = True)
        
    except AssertionError as error:
            print('An exception occurred: {}'.format(error))
            print(ind, row_play.playDescription)
            continue
    
    collected_stats.append(plaVstat)

dfstats = pd.concat(collected_stats)

print("--- {minutes:.0f}m {seconds:.0f}s ---".format(minutes = (time.time() - t_s)/60, seconds = (time.time() - t_s) % 60))

In [ ]:
dfstats.to_csv('Wk1to5Full.csv', index = False)